In [3]:
from cesnet_datazoo.datasets import CESNET_QUIC22
from cesnet_datazoo.config import DatasetConfig, AppSelection, ValidationApproach

dataset = CESNET_QUIC22("~/datasets/CESNET-QUIC22/", size="XS")

common_params = {
    "dataset": dataset,
    "apps_selection": AppSelection.ALL_KNOWN,
    "train_period_name": "W-2022-44",
    "val_approach": ValidationApproach.SPLIT_FROM_TRAIN,
    "train_val_split_fraction": 0.2,
    "use_packet_histograms": True,
}
dataset_config = DatasetConfig(**common_params)
dataset.set_dataset_config_and_initialize(dataset_config)
train_dataframe = dataset.get_train_df(flatten_ppi=True)
val_dataframe = dataset.get_val_df(flatten_ppi=True)
test_dataframe = dataset.get_test_df(flatten_ppi=True)

X = train_dataframe.drop(columns="APP").to_numpy()
y = train_dataframe["APP"].to_numpy()

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_jobs = -1)
clf.fit(X, y)

Loading data from dataloader


100%|██████████| 8162/8162 [00:08<00:00, 945.29it/s] 


Loading data from dataloader


100%|██████████| 192/192 [00:03<00:00, 48.62it/s]


Loading data from dataloader


100%|██████████| 1247/1247 [00:08<00:00, 143.58it/s]


RandomForestClassifier(n_jobs=-1)

In [14]:
import numpy as np

fi = clf.feature_importances_
fi_arr = []

for i, el in enumerate(fi):
    fi_arr.append((i, el))

fi_arr = sorted(fi_arr, key=lambda x: x[1], reverse=True)

for i in fi_arr:
    print(i)

(91, 0.049026129124047566)
(62, 0.039672063141830784)
(63, 0.0353087385610517)
(64, 0.032289335086405996)
(65, 0.027433553591366407)
(66, 0.026712175132183318)
(90, 0.02334581217729949)
(116, 0.022754563632823088)
(60, 0.022281908970251178)
(110, 0.022151761671259824)
(115, 0.021878022165601994)
(94, 0.02156386966699109)
(61, 0.019202658569022427)
(67, 0.01807762321995001)
(97, 0.01806561052896565)
(108, 0.015640199858541242)
(111, 0.015215633153027597)
(125, 0.014993252400284358)
(1, 0.014563566773058399)
(68, 0.014519501322159584)
(2, 0.014425318892024556)
(72, 0.01379973728739744)
(93, 0.01359993157013977)
(114, 0.013553933290455986)
(70, 0.012937305108541363)
(69, 0.012440915981442062)
(112, 0.012427787444249227)
(73, 0.01183555141871047)
(4, 0.011383360606351558)
(3, 0.011229132798905625)
(71, 0.01093358778794486)
(107, 0.010689821781752436)
(105, 0.010235159207444633)
(113, 0.009867704758064194)
(12, 0.009825889266930827)
(74, 0.009431361333164216)
(104, 0.00937186745800566)
(92,

In [21]:
for i, el in enumerate(train_dataframe.columns.tolist()):
    print(i, end = " ")
    print(el)

0 IPT_1
1 IPT_2
2 IPT_3
3 IPT_4
4 IPT_5
5 IPT_6
6 IPT_7
7 IPT_8
8 IPT_9
9 IPT_10
10 IPT_11
11 IPT_12
12 IPT_13
13 IPT_14
14 IPT_15
15 IPT_16
16 IPT_17
17 IPT_18
18 IPT_19
19 IPT_20
20 IPT_21
21 IPT_22
22 IPT_23
23 IPT_24
24 IPT_25
25 IPT_26
26 IPT_27
27 IPT_28
28 IPT_29
29 IPT_30
30 DIR_1
31 DIR_2
32 DIR_3
33 DIR_4
34 DIR_5
35 DIR_6
36 DIR_7
37 DIR_8
38 DIR_9
39 DIR_10
40 DIR_11
41 DIR_12
42 DIR_13
43 DIR_14
44 DIR_15
45 DIR_16
46 DIR_17
47 DIR_18
48 DIR_19
49 DIR_20
50 DIR_21
51 DIR_22
52 DIR_23
53 DIR_24
54 DIR_25
55 DIR_26
56 DIR_27
57 DIR_28
58 DIR_29
59 DIR_30
60 SIZE_1
61 SIZE_2
62 SIZE_3
63 SIZE_4
64 SIZE_5
65 SIZE_6
66 SIZE_7
67 SIZE_8
68 SIZE_9
69 SIZE_10
70 SIZE_11
71 SIZE_12
72 SIZE_13
73 SIZE_14
74 SIZE_15
75 SIZE_16
76 SIZE_17
77 SIZE_18
78 SIZE_19
79 SIZE_20
80 SIZE_21
81 SIZE_22
82 SIZE_23
83 SIZE_24
84 SIZE_25
85 SIZE_26
86 SIZE_27
87 SIZE_28
88 SIZE_29
89 SIZE_30
90 BYTES
91 BYTES_REV
92 PACKETS
93 PACKETS_REV
94 DURATION
95 PPI_LEN
96 PPI_ROUNDTRIPS
97 PPI_DURATION
98

In [53]:
from math import log

m1 = 0
m2 = 0

count1 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count2 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


for i in range(len(X)):
    count1[np.clip(int(log(X[i][91], 5)) - 4, 1, 7) - 1] += 1
    count2[np.clip(int(log(X[i][90], 3)) - 4, 2, 10) - 2] += 1

    # if m1 < max(int(log(X[i][91], 5)) - 4, 0):
    #     m1 = max(int(log(X[i][91], 5)) - 4, 0)
    
    # if m2 < max(int(log(X[i][90], 5)) - 4, 0):
    #     m2 = max(int(log(X[i][90], 5)) - 4, 0)

print(count1)
print(count2)

[1264327, 144476, 96905, 39280, 16213, 5247, 496, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[166569, 964554, 296540, 95292, 33069, 8583, 1467, 452, 237, 106, 75, 0, 0, 0, 0, 0, 0]
